In [ ]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [ ]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2113536/2110848 [==============================] - 0s 0us/step


In [ ]:
y_train.shape

(1395,)

In [ ]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

In [ ]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
print(X_train.shape)
print(X_test.shape)

(1395, 49, 1)
(599, 49, 1)


In [ ]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)
y_train = y_data[:1395]
y_test = y_data[1395:]

In [ ]:
print(y_train.shape)
print(y_test.shape)

(1395, 46)
(599, 46)


In [ ]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True,dropout=0.2))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False,dropout=0.2))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001) ## lr= learinng rate
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 50, batch_size = 50, verbose = 1,validation_data=(X_test,y_test))

In [ ]:
history=model.fit(X_train, y_train)

Epoch 1/50
28/28 [==============================] - 2s 88ms/step - loss: 2.4288 - accuracy: 0.5355 - val_loss: 1.3482 - val_accuracy: 0.7479
Epoch 2/50
28/28 [==============================] - 2s 79ms/step - loss: 1.3077 - accuracy: 0.7147 - val_loss: 1.1786 - val_accuracy: 0.7479
Epoch 3/50
28/28 [==============================] - 2s 77ms/step - loss: 1.1955 - accuracy: 0.7147 - val_loss: 1.1548 - val_accuracy: 0.7479
Epoch 4/50
28/28 [==============================] - 2s 75ms/step - loss: 1.1739 - accuracy: 0.7147 - val_loss: 1.1499 - val_accuracy: 0.7479
Epoch 5/50
28/28 [==============================] - 2s 75ms/step - loss: 1.1625 - accuracy: 0.7147 - val_loss: 1.1462 - val_accuracy: 0.7479
Epoch 6/50
28/28 [==============================] - 2s 89ms/step - loss: 1.1606 - accuracy: 0.7147 - val_loss: 1.1384 - val_accuracy: 0.7479
Epoch 7/50
28/28 [==============================] - 2s 76ms/step - loss: 1.1579 - accuracy: 0.7147 - val_loss: 1.1434 - val_accuracy: 0.7479
Epoch 8/50
28

In [ ]:
# Plot history: Categorical crossentropy & Accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Categorical crossentropy (training data)')
plt.plot(history.history['val_loss'], label='Categorical crossentropy (validation data)')
plt.plot(history.history['accuracy'], label='Accuracy (training data)')
plt.plot(history.history['val_accuracy'], label='Accuracy (validation data)')
plt.title('Model performance for StackedRNN example')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="middle right")
plt.show()

NameError: ignored